In [3]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.contrib import rnn

tf.reset_default_graph()
tf.set_random_seed(777)

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator / (denominator+1e-7)

xy = np.genfromtxt('/Users/yeseo/Desktop/taxi_data/City_Counted_TaxiMach_Link_Dataset_Full_201501 - 12.txt',delimiter = ',',dtype = None)

xy= xy[:,:27]
a = xy[:,:2]
b = xy[:,2:]
b= MinMaxScaler(b)
xy = np.hstack((a,b))

seq_length =6
data_dim =27
hidden_dim = 25
output_dim = 25
learning_rate = 0.01
iterations = 1001

train_size = int(len(xy)*0.7)
validation_size = int(len(xy)*0.2)

train_set = xy[:train_size]
validation_set = xy[train_size:train_size+validation_size]
test_set = xy[train_size+validation_size:]

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range (0,len(time_series)-seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i+seq_length,2:]
     
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

#train_set, test_set 만들기
trainX, trainY = build_dataset(train_set,seq_length)
validationX, validationY = build_dataset(validation_set,seq_length)
testX,testY = build_dataset(test_set, seq_length)

X = tf.placeholder(tf.float32,[None, seq_length,data_dim], name = "X")
Y = tf.placeholder(tf.float32,[None,25], name = "Y")

#LSTM CELL만들기
with tf.variable_scope("rnn"):
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple = True, activation=tf.tanh)
    outputs,_states = tf.nn.dynamic_rnn(cell,X,dtype = tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim,activation_fn = None)

    loss =tf.reduce_mean(tf.square(Y_pred-Y))
    train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

#RMSE 측정
targets = tf.placeholder(tf.float32,[None,25])
predictions = tf.placeholder(tf.float32,[None,25])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
        _, step_loss = sess.run([train,loss],feed_dict={X:trainX, Y:trainY})
        print("[step: {}] loss: {}".format(i,step_loss))
    
    save_path = saver.save(sess, "/Users/yeseo/ML_with_Taxidata/code/save_model_code/model/my_model.ckpt")
    
    test_predict = sess.run(Y_pred, feed_dict = {X:validationX})
    
    rmse_val = sess.run(rmse, feed_dict={targets: validationY,predictions: test_predict})
    
    print("RMSE: {} ".format(rmse_val))

[step: 0] loss: 0.22916409373283386
[step: 1] loss: 0.16104911267757416
[step: 2] loss: 0.11711139976978302
[step: 3] loss: 0.08661767095327377
[step: 4] loss: 0.0655556246638298
[step: 5] loss: 0.050411976873874664
[step: 6] loss: 0.04113463684916496
[step: 7] loss: 0.03385419026017189
[step: 8] loss: 0.03009498305618763
[step: 9] loss: 0.027184823527932167
[step: 10] loss: 0.02454414777457714
[step: 11] loss: 0.022536572068929672
[step: 12] loss: 0.021073197945952415
[step: 13] loss: 0.020195741206407547
[step: 14] loss: 0.018596967682242393
[step: 15] loss: 0.01751396618783474
[step: 16] loss: 0.016156751662492752
[step: 17] loss: 0.015727002173662186
[step: 18] loss: 0.015159176662564278
[step: 19] loss: 0.01455342024564743
[step: 20] loss: 0.014658471569418907
[step: 21] loss: 0.013619748875498772
[step: 22] loss: 0.013438323512673378
[step: 23] loss: 0.01246720552444458
[step: 24] loss: 0.012137931771576405
[step: 25] loss: 0.011427147313952446
[step: 26] loss: 0.0110337361693382

[step: 216] loss: 0.0022598837967962027
[step: 217] loss: 0.002278328873217106
[step: 218] loss: 0.002217439003288746
[step: 219] loss: 0.0022839675657451153
[step: 220] loss: 0.002250839490443468
[step: 221] loss: 0.0022127798292785883
[step: 222] loss: 0.002264333888888359
[step: 223] loss: 0.002201850526034832
[step: 224] loss: 0.0022113961167633533
[step: 225] loss: 0.0022134867031127214
[step: 226] loss: 0.002191923325881362
[step: 227] loss: 0.0021826718002557755
[step: 228] loss: 0.0021869658958166838
[step: 229] loss: 0.0021774820052087307
[step: 230] loss: 0.002156891394406557
[step: 231] loss: 0.002171956468373537
[step: 232] loss: 0.0021579409949481487
[step: 233] loss: 0.002141704550012946
[step: 234] loss: 0.0021533865947276354
[step: 235] loss: 0.002141553210094571
[step: 236] loss: 0.002129362663254142
[step: 237] loss: 0.0021339519880712032
[step: 238] loss: 0.002123965648934245
[step: 239] loss: 0.0021216163877397776
[step: 240] loss: 0.0021176880691200495
[step: 241] 

[step: 426] loss: 0.0018468416528776288
[step: 427] loss: 0.0018554345006123185
[step: 428] loss: 0.0017953369533643126
[step: 429] loss: 0.0016767618944868445
[step: 430] loss: 0.0016181806568056345
[step: 431] loss: 0.0016014951979741454
[step: 432] loss: 0.0016513486625626683
[step: 433] loss: 0.0016783202299848199
[step: 434] loss: 0.0017041877144947648
[step: 435] loss: 0.001662593800574541
[step: 436] loss: 0.0016639282694086432
[step: 437] loss: 0.0016650699544698
[step: 438] loss: 0.0016017014859244227
[step: 439] loss: 0.0015744863776490092
[step: 440] loss: 0.0016318721463903785
[step: 441] loss: 0.0016117618652060628
[step: 442] loss: 0.0015853274380788207
[step: 443] loss: 0.0015991973923519254
[step: 444] loss: 0.001601541880518198
[step: 445] loss: 0.0015638954937458038
[step: 446] loss: 0.0015549898380413651
[step: 447] loss: 0.0015769472811371088
[step: 448] loss: 0.0015540665481239557
[step: 449] loss: 0.0015401220880448818
[step: 450] loss: 0.0015505686169490218
[step

[step: 636] loss: 0.0010566558921709657
[step: 637] loss: 0.0010448717512190342
[step: 638] loss: 0.0010433330899104476
[step: 639] loss: 0.0010482316138222814
[step: 640] loss: 0.0010570118902251124
[step: 641] loss: 0.001066853990778327
[step: 642] loss: 0.0010815846035256982
[step: 643] loss: 0.0010914960876107216
[step: 644] loss: 0.0011083773570135236
[step: 645] loss: 0.0011060165707021952
[step: 646] loss: 0.0011066475417464972
[step: 647] loss: 0.0010909934062510729
[step: 648] loss: 0.0010805731872096658
[step: 649] loss: 0.0010666247690096498
[step: 650] loss: 0.001052709762006998
[step: 651] loss: 0.0010366588830947876
[step: 652] loss: 0.0010219081304967403
[step: 653] loss: 0.0010128453141078353
[step: 654] loss: 0.001010125852189958
[step: 655] loss: 0.0010099868522956967
[step: 656] loss: 0.001009350293315947
[step: 657] loss: 0.0010069760028272867
[step: 658] loss: 0.0010054833255708218
[step: 659] loss: 0.0010076977778226137
[step: 660] loss: 0.0010156307835131884
[ste

[step: 846] loss: 0.0008396649500355124
[step: 847] loss: 0.0008433747570961714
[step: 848] loss: 0.0008560072164982557
[step: 849] loss: 0.0008648925577290356
[step: 850] loss: 0.0008628757786937058
[step: 851] loss: 0.0008544012671336532
[step: 852] loss: 0.0008527419995516539
[step: 853] loss: 0.0008402870735153556
[step: 854] loss: 0.0008226358913816512
[step: 855] loss: 0.0008007965516299009
[step: 856] loss: 0.0007904600352048874
[step: 857] loss: 0.0007911825668998063
[step: 858] loss: 0.0007948575657792389
[step: 859] loss: 0.0007991709862835705
[step: 860] loss: 0.0008033637423068285
[step: 861] loss: 0.0008107558242045343
[step: 862] loss: 0.0008172052330337465
[step: 863] loss: 0.0008280651527456939
[step: 864] loss: 0.0008494883659295738
[step: 865] loss: 0.000885836489032954
[step: 866] loss: 0.0009332750923931599
[step: 867] loss: 0.000959084602072835
[step: 868] loss: 0.0009393020300194621
[step: 869] loss: 0.0008778161136433482
[step: 870] loss: 0.000844032852910459
[st

In [6]:
test_predict

array([[0.519983  , 0.5357133 , 0.7311876 , ..., 0.62834936, 0.8134205 ,
        0.77985823],
       [0.48948288, 0.5064425 , 0.7059424 , ..., 0.59972465, 0.8103568 ,
        0.7986638 ],
       [0.46200874, 0.4708667 , 0.6635073 , ..., 0.5821145 , 0.80588466,
        0.7984813 ],
       ...,
       [0.65016776, 0.6609901 , 0.62372655, ..., 0.61395335, 0.3835152 ,
        0.27562097],
       [0.6763084 , 0.6765678 , 0.64603007, ..., 0.64307237, 0.43519932,
        0.31744656],
       [0.69684863, 0.6880646 , 0.6709062 , ..., 0.67722094, 0.46239683,
        0.3378895 ]], dtype=float32)